# 5장 서포트 벡터 머신
## EPOCH 3기 한우림

- 서포트 벡터 머신(Support Vector Machine; SVM): 매우 강력하고 선형/비선형 분류, 회귀, 이상치 탐색에 사용할 수 있는 다목적 머신러닝 모델
- 복잡한 분류 문제에 잘 들어맞으며 작거나 중간 크기의 데이터셋에 적합

### 5.1 선형 SVM 분류

- SVM: 클래스 사이에 가장 폭이 넓은 도로를 찾는 것 -> 라지 마진 분류(large margin classification)
  - SVM은 특성의 스케일에 민감
- 도로 경계에 위치한 샘플에 의해 전적으로 결정 또는 의지됨 -> 이런 샘플을 서포트 벡터라고 함

#### 5.1.1 소프트 마진 분류
- 하드 마진 분류: 모든 샘플이 도로 바깥쪽에 올바르게 분류
  - 문제점: 데이터가 선형적으로 구분될 수 있어야 하며 이상치에 민감
=> 도로의 폭을 가능한 한 넓게 유지하는 것과 마진 오류 사이에 적절한 균형을 잡아야 함 --> 소프트 마진 분류
- C: 사이킷런의 SVM 모델을 만들 때 여러 하이퍼파라미터 중 하나

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

# LinearSVC 대신 선형 커널을 사용하는 SVC 클래스로 대체 가능 -> 일반적 확률적 경사 하강법 적용
# LinearSVC만큼 빠르게 수렴하지는 않지만 데이터셋이 아주 커서 메모리에 적재할 수 없거나 온라인 학습으로 분류 문제를 다룰 때 유용

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 너비
y = (iris["target"] == 2).astype(np.float64)  # Iris virginica

svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42)),
    ])

svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge', random_state=42))])

In [ ]:
svm_clf.predict([[5.5,1.7]])

array([1.])

### 5.2 비선형 SVM 분류
- 선형적으로 분류할 수 없는 데이터셋이 많음 -> 다항 특성과 같은 특성을 더 추가

In [ ]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3)),
        ("scaler", StandardScaler()),
        ("svm_clf", LinearSVC(C=10, loss="hinge", random_state=42))
    ])

polynomial_svm_clf.fit(X, y)

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge', random_state=42))])

#### 5.2.1 다항식 커널
- 낮은 차수의 다항식: 매우 복잡한 데이터셋을 잘 표현하지 못함
- 높은 차수의 다항식: 많은 특성 추가 -> 모델 속도 감소
- 커널 트릭(kernel trick): 실제로는 특성을 추가하지 않으면서 다항식 특성을 많이 추가한 것과 같은 결과를 가져옴

In [ ]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
    ])
poly_kernel_svm_clf.fit(X, y)

# 3차 다항식 커널로 SVM 훈련
# 모델이 과대적합인 경우, 다항식의 차수를 줄이고 과소적합인 경우, 차수를 늘려야 함

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

#### 5.2.2 유사도 특성
- 유사도 함수: 각 샘플이 특정 랜드마크와 얼마나 닮았는지 측정하여 특성 추가
- 가우시안 RBF (방사 기저 함수) 정의
  - 0(랜드마크에서 아주 멀리 떨어진 경우) ~ 1(랜드마크와 같은 위치일 경우)
  - 종 모양
- 랜드마크 선택 방법: 데이터셋에 있는 모든 샘플 위치에 설정 -> 차원이 매우 커지고 변환된 훈련 세트가 선형적으로 구분될 가능성 높음
  - 훈련 세트에서 n개의 특성을 가진 m개의 샘플이 m개의 특성을 가진 m개의 샘플로 변환됨

#### 5.2.3 가우시안 RBF 커널

In [ ]:
rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])
rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

- gamma를 증가시키면 종 모양 그래프가 좁아져서 각 샘플의 영향 범위가 작아짐 -> 결정 경계가 더 불규칙, 각 샘플을 따라 구불구불하게 휨
- 작은 gamma: 넓은 종 모양 그래프 -> 결정 경계가 부드러워짐
=> 즉, hyper parameter gamma: 규제의 역할
  - 과대적합일 경우 감소
  - 과소적합일 경우 증가

#### 5.2.4 계산 복잡도
- liblinear 라이브러리: 커널 트릭 지원 x, 훈련 샘플과 특성 수에 거의 선형적으로 증가
  - 훈련 시간 복잡도: O(mxn)
  - 정밀도를 높이면 알고리즘의 수행 시간이 길어짐 -> 허용오차 하이퍼파라미터 epsilon으로 조절
- libsvm 라이브러리: 커널 트릭 알고리즘 구현
  - 훈련 시간 복잡도: O(m^2\*n), O(m^3\*n) 사이
  - 훈련 샘플 수가 커지면 속도가 느려짐
  - 복잡하지만 작거나 중간 규모의 훈련 세트에 적합
  - 희소 특성인 경우 잘 확장됨

### 5.3 SVM 회귀
- SVM 회귀: 제한된 마진 오류 안에서 도로 안에 가능한 한 많은 샘플이 들어가도록 학습
- 도로의 폭: 하이퍼파라미터 epsilon으로 조절
- 마진 안에서는 훈련 샘플이 추가되어도 모델의 예측에는 영향 x
- epsilon에 민감하지 않다고 함

In [ ]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X, y)

LinearSVR(epsilon=1.5, random_state=42)

In [ ]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1, gamma="scale")
svm_poly_reg.fit(X, y)

# SVR: SVC의 회귀 버전, 훈련 세트가 커지면 훨씬 느려짐
# LinearSVR: LinearSVC의 회귀 버전, 필요한 시간이 훈련 세트의 크기에 비례해서 선형적으로 증가

SVR(C=100, degree=2, kernel='poly')

### 9번 연습문제
- MNIST 데이터셋에 SVM 분류기 훈련
- OvR 전략을 사용해 10개의 숫자를 분류
- 처리 속도를 높이기 위해 작은 검증 세트로 하이퍼파라미터를 조정하는 것이 좋음

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [ ]:
X, y = mnist["data"], mnist["target"]
print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


In [ ]:
# train, test split (6만, 1만장씩)
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
# 선형 SVM

import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

# LinearSVC 대신 선형 커널을 사용하는 SVC 클래스로 대체 가능 -> 일반적 확률적 경사 하강법 적용
# LinearSVC만큼 빠르게 수렴하지는 않지만 데이터셋이 아주 커서 메모리에 적재할 수 없거나 온라인 학습으로 분류 문제를 다룰 때 유용

svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42)),
    ])

svm_clf.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge', random_state=42))])

In [ ]:
# 정확도 측정
from sklearn.metrics import accuracy_score
pred1 = svm_clf.predict(X_train)
accuracy_score(y_train, pred1)

0.9298333333333333

In [ ]:
# 비선형 SVM
from sklearn.preprocessing import PolynomialFeatures

polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=2)),
        ("scaler", StandardScaler()),
        ("svm_clf", LinearSVC(C=3, loss="hinge", random_state=42))
    ])

polynomial_svm_clf.fit(X_train, y_train)

비선형 SVM 학습 중 RAM 초과됨...... gpt가 말하는 문제점과 해결 방법

🔍 RAM 사용량이 높은 이유

1. MNIST 데이터 크기
- X의 크기는 (70000, 784)로, 6만 개의 훈련 데이터와 1만 개의 테스트 데이터를 포함함.
- fetch_openml(..., as_frame=False)로 데이터를 numpy 배열로 가져왔기 때문에 RAM을 많이 사용함.

2. LinearSVC의 데이터 변환 방식
- LinearSVC는 기본적으로 전체 데이터를 한 번에 메모리에 올려서 처리하기 때문에, 고해상도 이미지 데이터에서는 RAM이 초과될 수 있음.
- SVC(kernel="linear")와 달리 LinearSVC는 배치 경사 하강법(SGD) 기반 최적화를 사용하지만, 큰 데이터셋을 한 번에 학습할 경우 메모리 문제가 발생할 수 있음.

3. 다항 특성 변환 (PolynomialFeatures)
- 다항 변환은 특성 수를 기하급수적으로 증가시킴.
- degree=2만 해도 (784, 784+307,080)이 되어 약 30만 개의 특성이 생성됨.
이를 StandardScaler와 LinearSVC에 적용하면 RAM이 급격히 증가할 수 있음.


✅ 해결 방법

1. 데이터 샘플링
- 하이퍼파라미터 조정을 위해 전체 6만 개 데이터를 사용할 필요 없음.
- train_test_split을 사용해 일부 데이터를 샘플링하여 훈련 속도를 높이고 RAM 사용량을 줄일 수 있음.

2. LinearSVC 대신 SGDClassifier 사용
- SGDClassifier(loss="hinge")는 LinearSVC와 동일한 선형 SVM이지만, **미니배치 확률적 경사 하강법(SGD)**을 사용하여 훨씬 적은 메모리를 사용함.
partial_fit()을 사용하면 온라인 학습도 가능함.

3. 다항 특성 변환 제거 또는 축소
- PolynomialFeatures(degree=2)를 사용할 경우, RAM이 과부하될 수 있으므로 제거하거나, 샘플 크기를 줄이는 것이 좋음.

4. 데이터 정규화 및 형 변환 최적화
- fetch_openml에서 불러온 y는 문자열이므로, 이를 정수로 변환하여 불필요한 메모리 사용을 줄일 수 있음.
- astype(np.uint8)을 사용하면 더 적은 메모리를 사용할 수 있음.


=> 내가 선택한 해결 방법: 데이터셋의 일부만 사용 (7만개가 아니라... 2만개 정도?)

In [3]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"].astype(np.uint8)

# 데이터 샘플링 및 정규화(2.5.4 특성 스케일링 참고)
X_train, X_test, y_train, y_test = train_test_split(X[:20000], y[:20000], test_size=0.2, random_state=42)  # 20,000개만 사용

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 선형 SVM c=1

svm_clf = LinearSVC(C=1, loss="hinge", random_state=42)
svm_clf.fit(X_train_scaled, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge', random_state=42))])

In [ ]:
# 정확도 측정
from sklearn.metrics import accuracy_score
y_train_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_train_pred)

0.946875

In [ ]:
# 선형 SVM c=2

svm_clf2 = LinearSVC(C=2, loss="hinge", random_state=42)
svm_clf2.fit(X_train_scaled, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=2, loss='hinge', random_state=42)

In [ ]:
# 정확도 측정
y_train_pred2 = svm_clf2.predict(X_train_scaled)
accuracy_score(y_train, y_train_pred2)

0.956125

정확도가 너무 높은 게 아닌가........... -> 과대적합 가능성?

In [4]:
# 비선형 SVM
from sklearn.preprocessing import PolynomialFeatures

polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=2)),
        ("svm_clf", LinearSVC(C=1, loss="hinge", random_state=42))
    ])

polynomial_svm_clf.fit(X_train_scaled, y_train)

# 정확도 측정
y_train_pred3 = polynomial_svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_train_pred3)

InvalidParameterError: The 'penalty' parameter of LinearSVC must be a str among {'l2', 'l1'}. Got PolynomialFeatures() instead.

비선형 학습 중 계속 세션 다운됨 -> 코랩 무료 버전에서 할당되는 RAM으로는 MNIST 데이터셋에 대해 다항 특성 추가가 어려울 것 같다......

### 10번 연습문제
- 캘리포니아 주택 가격 데이터셋에 SVM 회귀 훈련

In [5]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import model_selection
from sklearn import metrics

cali = datasets.fetch_california_housing()
print(cali)

X = cali['data']
y = cali['target']

'''
df = pd.DataFrame(cali.data, columns=cali.feature_names)
df['target'] = cali.target
df.head()
'''

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]]), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset:\n\nCalifornia Housing dataset\n-

"\ndf = pd.DataFrame(cali.data, columns=cali.feature_names)\ndf['target'] = cali.target\ndf.head()\n"

In [9]:
print(len(X))
print(len(y))
print()

20640
20640
[[   8.3252       41.            6.98412698 ...    2.55555556
    37.88       -122.23      ]
 [   8.3014       21.            6.23813708 ...    2.10984183
    37.86       -122.22      ]
 [   7.2574       52.            8.28813559 ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.7          17.            5.20554273 ...    2.3256351
    39.43       -121.22      ]
 [   1.8672       18.            5.32951289 ...    2.12320917
    39.43       -121.32      ]
 [   2.3886       16.            5.25471698 ...    2.61698113
    39.37       -121.24      ]]


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X_train_scaled, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVR(epsilon=1.5, random_state=42)

In [18]:
# 회귀 지표 평균제곱오차(mse) 측정

from sklearn.metrics import mean_squared_error

y_train_pred = svm_reg.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_train_pred)
print(mse)

0.78458307932599


In [19]:
np.sqrt(mse)

np.float64(0.8857669441371077)

In [20]:
# SVR: SVC의 회귀 버전, 훈련 세트가 커지면 훨씬 느려짐
# LinearSVR: LinearSVC의 회귀 버전, 필요한 시간이 훈련 세트의 크기에 비례해서 선형적으로 증가

from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1, gamma="scale")
svm_poly_reg.fit(X_train_scaled, y_train)

SVR(C=100, degree=2, kernel='poly')

In [21]:
y_train_pred = svm_poly_reg.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_train_pred)
print(mse)

1.3096082711120771


- 다항회귀 mse가 더 높은 결과...
- Grid search or randomized search로 하이퍼파라미터 조정해 보면 좋을 듯